# Registration and tracking: optical flow

### 1. Overview & learning objectives
In this notebook we will learn about registration and tracking, two sides of one common idea. 

With this notebook we will:

1. Learn how to open, display, and interact with 3D images in Jupyter notebooks.

1. Use optical flow to register consecutive slices on an image identifying key parameters in the registration process.

1. Calculate and visualize optical flow vector fields for subsequent application in tracking. 

### 2. Loading and browsing 3D images
Registration and tracking are two applications of one concept: in image stacks containing multiple slices of a certain sample (Z planes, time points, colours, imaging modalities), it is possible to identify a transformation that maps one slice (or the objects on that slice) onto the next.

Our first step to learn about registration and tracking is to open an image stack:

In [ ]:
from skimage import io

# Read image from disk.
astack = io.imread('cells_movie.tif')


Take a look a the dimensions of *astack*: how is this different from other images that we used before?

In [ ]:
# DELETE THIS.
# Inspect the shape: this is not a 2D image any longer.
astack.shape

We will define a helper class to visualize 3D images. This is also a nice example of how to use the **ipywidgets** package to create interactive notebooks.

Run the cell below and then use this class to visualize *astack*:

In [ ]:
import numpy as np
import ipywidgets as ipyw
import matplotlib.pyplot as plt
%matplotlib inline

class ImageSliceViewer3D:
    """ 
    ImageSliceViewer3D displays volumetric data. 
    
    User can interactively change the slice plane and 
    the slice number being viewed. 

    Arguments:
    volume = 3D input image
    figsize = default(8,8), to set the size of the figure
    cmap = default('gray'), string for the matplotlib colormap. You can find 
    more matplotlib colormaps on the following link:
    https://matplotlib.org/2.0.2/users/colormaps.html
    
    """
    
    def __init__(self, volume, figsize=(8,8), cmap='gray'):
        self.volume = volume
        self.figsize = figsize
        self.cmap = cmap
        plt.set_cmap(cmap)
        self.v = [np.min(volume), np.max(volume)]
        
        # Create interactive widget.
        ipyw.interact(self.view_selection, view=ipyw.RadioButtons(
            options=['x-y','z-y', 'z-x'], value='x-y', 
            description='slice plane selection:', disabled=False,
            style={'description_width': 'initial'}))
    
    def view_selection(self, view):
        # Transpose the volume to orient according to the slice plane.
        orient = {"x-y": [0, 1, 2], "z-x": [1, 2, 0], "z-y": [2, 1, 0]}
        self.vol = np.transpose(self.volume, orient[view])
        
        # Create slider to change slice and link it to the function that plots the slice.
        maxZ = self.vol.shape[0] - 1
        ipyw.interact(self.plot_slice, 
            z=ipyw.IntSlider(min=0, max=maxZ, step=1, continuous_update=True, 
            description='Image Slice:'))
        
    def plot_slice(self, z):
        # Plot slice for the given plane and slice.
        self.fig = plt.figure(figsize=self.figsize)
        plt.imshow(self.vol[z,:,:], vmin=self.v[0], vmax=self.v[1])



In [ ]:
# DELETE.
ImageSliceViewer3D(astack);  # semi-colon suppresses text output

### 3. Registration
In **registration** we use the transformation to align consecutive slices. Here, we will use the **optical flow** to estimate the change in the signal across consecutive slices. We will use *optical_flow_tvl1* in the **skimage.registration** module to calculate the optical flow, and the *warp* function in **skimage.transform** to apply the optical flow field to consecutive slices.

First, take a look at the help for both functions and identify their key parameters:

In [ ]:
# DELETE THIS CODE
from skimage import registration as skr

skr.optical_flow_tvl1?

In [ ]:
# DELETE THIS CODE
from skimage import transform as skt

skt.warp?

Now fill in the parameters in the calls to *optical_flow_tvl1* and *warp* below:

In [ ]:
import numpy as np

from skimage import registration as skr
from skimage import transform as skt

# DELETE THIS CODE.
## this here is a toy example to better understand the effect of warp.
# astack = astack[0:2]
# row_coords, col_coords = np.meshgrid(np.arange(
#         astack.shape[1]), np.arange(astack.shape[2]), indexing='ij')
#
## warp applies the inverse transformation!!
# astack[1] = skt.warp(astack[0], numpy.array([row_coords + 2, col_coords + 2]), preserve_range=True)    

# create array to store registered image.
registered_stack_tvl1 = np.empty(astack.shape)
registered_stack_tvl1[0] = astack[0]

# create coordinate matrices to use in warp.
row_coords, col_coords = np.meshgrid(np.arange(astack.shape[1]), np.arange(astack.shape[2]), indexing='ij')

# compute the optical flow: for every slice in the stack
for slice_index in range(astack.shape[0]-1): 
    # compute the optical flow between the current and the next slice: 
    # this returns the transformation that converts the reference image into the moving image.
    # DELETE THIS LINE (the stuff in parentheses)
    v, u = skr.optical_flow_tvl1(astack[slice_index], astack[slice_index+1])

    # apply optical flow and store transformed image: remember warp applies the inverse transformation!
    # DELETE THIS LINE (the stuff in parentheses)
    registered_stack_tvl1[slice_index+1] = skt.warp(astack[slice_index+1], np.array([row_coords + v, col_coords + u]), order=5, mode='constant', cval=0, preserve_range=True)


Use our helper class above to visualize the registration results in *registered_stack*. What happened? How has the image changed? What is the effect of the *order* and *mode* parameters for *warp*?

In [ ]:
# DELETE THIS CODE
ImageSliceViewer3D(registered_stack_tvl1)  

# order determines the type of interpolation applied: the greater the order, 
# the better the results (compare to order 0, nearest neighbour);
# mode defines what values are used for padding 
# (e.g. 'edges' repeats the values around the image edge)


### 4. Tracking

In **tracking** the transformation is used to match object positions across slices and reconstruct their trajectories. We will now use the Lucas-Kanade algorithm implemented in *optical_flow_ilk* in the **skimage.registration** package. Take a look at the help for *optical_flow_ilk*, paying attention to the *radius* parameter.

In [ ]:
# DELETE THIS CODE
skr.optical_flow_ilk?

We will now use *optical_flow_ilk* to create and visualize the optical flow field across slices. Interpolating the value of the flow field at any point **p** of the image generates a vector **v** such that **p** + **v** predicts the position of **p** in the next slice. As you can imagine, this is at the base of several tracking algorithms.

Examine the code below and complete the code to invoke *optical_flow_ilk*. 

In [ ]:
# configuration for quiver plots
nvec = 20  # number of vectors to be displayed along each image dimension
nr, nc = astack.shape[1:]
step = max(nr//nvec, nc//nvec)
flow_stack = np.empty((astack.shape[0]-1, 576, 576))  # we'll display figures in 8x8 inches, with 72 pixels/inch.

# compute the optical flow: for every slice in the stack
for slice_index in range(astack.shape[0]-1): 
    # compute the optical flow between the current and the next slice.
    # DELETE THIS LINE (the stuff in parentheses)
    v, u = skr.optical_flow_ilk(astack[slice_index], astack[slice_index+1], radius=7)

    # display optical flow
    # compute flow magnitude
    flow_magnitude = np.sqrt(u ** 2 + v ** 2)

    y, x = np.mgrid[:nr:step, :nc:step]
    u_ = u[::step, ::step]
    v_ = v[::step, ::step]

    # create a figure and display the vector field overlaid on an image.
    # plt.imshow(flow_magnitude)  # use this instead of next line if you want to visualize the flow magnitude. 
    fig = plt.figure(figsize=(8, 8), frameon=False, tight_layout=True)
    plt.imshow(astack[slice_index])  # use this instead of the line above to visualize the original image. 
    plt.quiver(x, y, u_, v_, color='r', units='dots', angles='xy', scale=0.25, scale_units='xy', lw=3)
    plt.gca().set_axis_off()

    # capture figure contents into flow_stack
    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    flow_stack[slice_index] = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))[:, :, 0]

    # close the figure
    plt.close(fig)



Use our helper class to visualize *flow_stack*. What happens if you use different values for the *radius* parameter? 

In [ ]:
# DELETE THIS CODE
ImageSliceViewer3D(flow_stack)

# radius values that are too small (e.g. 3) lead to  disorganized vector fields;
# radius values that are too large (e.g. 128) lead to a uniform vector field;

Now consider how one may use optical flow fields for cell tracking: if we interpolate the optical flow field at the centroid of objects detected in an image, we can predict the position of the objects in the next slice, and identify corresponding objects across slices. Other approaches that integrate segmentation and tracking are also possible, for instance by applying the optical flow to the seeds for watershed segmentation on one slice to identify seeds on the following slices. Objects segmented from matching seeds represent the same object in different slices.  